In [1]:
%matplotlib


import io
import os
import sys
import numpy as np
from astropy.io import fits
from os.path import basename
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from copy import deepcopy as copy

Using matplotlib backend: agg


In [2]:
from lvmdap.modelling.ingredients import StellarModels
from lvmdap.modelling.synthesis import StellarSynthesis
from lvmdap._cmdline import dap

In [3]:
# orion distance in kpc
orion_distance = 412/1000
# M31 distance in kpc
M31_distance = 778

import astropy.units as u
from astropy.cosmology import z_at_value

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
cosmo

orion_z = z_at_value(cosmo.luminosity_distance, orion_distance*u.kpc)
m31_z = z_at_value(cosmo.luminosity_distance, M31_distance*u.kpc)

print(f"Orion Nebula z ~ {orion_z}")
print(f"M31 z          ~ {m31_z}")

Orion Nebula z ~ 9.635695717842103e-08
M31 z          ~ 0.00018163451556510232


In [4]:
basis = "../../_fitting-data/_basis/stellar-basis-spectra-10-miles-indous.fits.gz"

# spectrum = "tests/NGC5947.spec_5.txt"
spectrum = "../../_fitting-data/spectra/orion_spec.txt"


orion_delta = orion_z/100
orion_ini = orion_z-10*orion_delta
orion_fin = orion_z+10*orion_delta

m31_delta = m31_z/100
m31_ini = m31_z-10*m31_delta
m31_fin = m31_z+10*m31_delta

# config = "../../_fitting-data/_configs/auto.NGC5947.config"
config = "../../_fitting-data/_configs/auto_ssp_V500_several_Hb.config"
mask_elines = "../../_fitting-data/_configs/mask_elines.txt"
emission_lines = "../../_fitting-data/_configs/emission_lines.txt"

label = "test_out"

In [5]:
sed = np.genfromtxt(spectrum)

# if "orion" in spectrum:
#     sed[:,2] *= 1e-16
#     name = spectrum.split(".")
#     spectrum = f"{name[0]}-fixed.{name[1]}"
#     np.savetxt(spectrum, sed)

# plt.plot(sed[:,1], sed[:,2])

# np.sqrt((10/2.355)**2 - (2.63/2.355)**2),np.sqrt((6.5/2.355)**2 - (2.63/2.355)**2)

In [7]:
cmdlike_ngc5947 = f"{spectrum} {basis},{basis},0.84 {label} {mask_elines} {config} 1 -3 150 3800,3850 7000,4700 {emission_lines} 0.0196 0.0001 0.0186 0.0206 30 20 1 400 0.3 0.15 0.0 1.6".split()
cmdlike_orion = f"{spectrum} {basis},{basis},3.34 {label} {mask_elines} {config} 0 -3 150 3850,3850 6750,4700 {emission_lines} {orion_z} {orion_delta} {orion_ini} {orion_fin} 30 20 1 1000 0.3 0.15 0.0 4.6".split()
cmdlike_m31 = f"{spectrum} {basis},{basis},0.84 {label} {mask_elines} {config} 0 -3 150 3800,3850 7000,4700 {emission_lines} {m31_z} {m31_delta} {m31_ini} {m31_fin} 30 20 1 1000 0.3 0.15 0.0 3.6".split()

In [8]:
print(" ".join(cmdlike_orion))

../../_fitting-data/spectra/orion_spec.txt ../../_fitting-data/_basis/stellar-basis-spectra-10-miles-indous.fits.gz,../../_fitting-data/_basis/stellar-basis-spectra-10-miles-indous.fits.gz,3.34 test_out ../../_fitting-data/_configs/mask_elines.txt ../../_fitting-data/_configs/auto_ssp_V500_several_Hb.config 0 -3 150 3850,3850 6750,4700 ../../_fitting-data/_configs/emission_lines.txt 9.635695717842103e-08 9.635695717842103e-10 8.672126146057892e-08 1.0599265289626313e-07 30 20 1 1000 0.3 0.15 0.0 4.6


In [9]:
dap._main(cmdlike_orion)

# move outputs to tests/
!mv *{label}* ../../tests/.

usage: ipykernel_launcher.py [-h] [--emission-lines-file EMISSION_LINES_FILE] [--rsp-nl-file RSP_NL_FILE] [--plot PLOT] [--flux-scale min max] [--w-range wmin wmax]
                             [--w-range-nl wmin2 wmax2] [--redshift input_redshift delta_redshift min_redshift max_redshift] [--sigma input_sigma delta_sigma min_sigma max_sigma]
                             [--AV input_AV delta_AV min_AV max_AV] [--ext-curve {CCM,CAL}] [--RV RV] [--single-rsp] [--n-mc N_MC] [-o path] [-c] [-v] [-d]
                             spectrum-file rsp-file sigma-inst label mask-file config-file
ipykernel_launcher.py: error: argument sigma-inst: invalid float value: 'test_out'
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.8/argparse.py", line 2434, in _get_value
    result = type_func(arg_string)
ValueError: could not convert string to float: 'test_out'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.8/argparse.py", line 1812, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/usr/lib/python3.8/argparse.py", line 2021, in _parse_known_args
    stop_index = consume_positionals(start_index)
  File "/usr/lib/python3.8/argparse.py", line 1977, in consume_positionals
    take_action(action, args)
  File "/usr/lib/python3.8/argparse.py", line 1870, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "/usr/lib/python3.8/argparse.py", line 2401, in _get_values
    value = self._get_value(action, arg_string)
  File "/usr/lib/python3.8/argparse.py", line 2447, in _get_value
    raise ArgumentError(ac

TypeError: object of type 'NoneType' has no len()

In [ ]:
!cat tests/elines_test_out